In [1]:
from kinconnect_api.config import MONGO_CONNECTION_STRING, load_dotenv
import os
from pymongo import MongoClient
import os, pymongo, pprint
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pymongo import MongoClient

# Connect to your Atlas cluster
client = MongoClient(MONGO_CONNECTION_STRING)
# Define collection and index name
db_name = "kinconnect"
collection_name = "app"
atlas_collection = client[db_name][collection_name]
vector_search_index = "vector_index"

2024-06-14 14:57:45.378 | INFO     | kinconnect_api.config:<module>:15 - PROJ_ROOT path is: /Users/nehiljain/code/kinconnect/kinconnect_api


In [2]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi


uri = MONGO_CONNECTION_STRING
# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [3]:
loader = PyPDFLoader("https://query.prod.cms.rt.microsoft.com/cms/api/am/binary/RE4HkJP")
data = loader.load()
# Split PDF into documents
from langchain_text_splitters import MarkdownHeaderTextSplitter
markdown_document = "# Foo\n\n    ## Bar\n\nHi this is Jim\n\nHi this is Joe\n\n ### Boo \n\n Hi this is Lance \n\n ## Baz\n\n Hi this is Molly"

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
md_header_splits = markdown_splitter.split_text(markdown_document)
md_header_splits
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
docs = text_splitter.split_documents(data)
# Print the first document
docs[0]

Document(page_content='Mong oDB Atlas Best P racticesJanuary 20 19A MongoD B White P aper', metadata={'source': 'https://query.prod.cms.rt.microsoft.com/cms/api/am/binary/RE4HkJP', 'page': 0})

In [6]:
from langchain_fireworks import FireworksEmbeddings
load_dotenv()


# Create the vector store
vector_search = MongoDBAtlasVectorSearch.from_documents(
    documents = docs,
    embedding = FireworksEmbeddings(model="nomic-ai/nomic-embed-text-v1.5"),
    collection = atlas_collection,
    index_name = vector_search_index
)

BadRequestError: Number of rows exceed limit of 256

In [7]:
len(docs)

424